In [77]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from itertools import product, chain
import importlib

import trainer, data, models

In [85]:
importlib.reload(trainer)
importlib.reload(data)
importlib.reload(models)

<module 'models' from '/Users/ralph/projects/d2dl/models.py'>

In [86]:
# load dataset 
dataset = data.fashion_mnist()

In [87]:
class LeNet(nn.Sequential):

    def __init__(self, output_classes):
        super().__init__(
            nn.LazyConv2d(out_channels=6, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(out_channels=16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(120),
            nn.ReLU(),
            nn.LazyLinear(84),
            nn.ReLU(),
            nn.LazyLinear(output_classes),
        )

In [88]:
provider = lambda hyperparams: trainer.Trainer(
    # swap with your model
    model=LeNet(10),
    dataset=dataset,
    # swap with your loss
    loss=nn.CrossEntropyLoss,
    opt=torch.optim.SGD,
    hyperparameters=hyperparams
)

In [89]:
# swap hyperparams here
model_trainer = provider(trainer.Hyperparameters(
    opt=dict(
        lr = 3e-3
    ),
    model=dict(),
    general=dict(
        num_epochs = 10,
        batch_size = 64
    )
))

In [128]:
dx = 4
ctxlen = 10
batch_size = 7
hidden_state_dim = 4
output_dim = 8

x = torch.rand((batch_size, ctxlen, dx))
w_xh = torch.rand((dx, hidden_state_dim))
h = torch.rand(hidden_state_dim)
w_hh = torch.rand((hidden_state_dim, hidden_state_dim))
w_hq = torch.rand((hidden_state_dim, output_dim))
x_prime = torch.movedim(x, 1, 0)
a = nn.ReLU()
outputs = []
for t, x_t in enumerate(x_prime):
    h_t1 = a(torch.matmul(x_t, w_xh) + torch.matmul(h, w_hh))
    h = h_t1
    
    output = a(torch.matmul(h_t1, w_hq))
    outputs.append(output)
torch.stack(outputs).shape

In [98]:
w_hh = torch.rand((hidden_state_dim, hidden_state_dim))

In [100]:
torch.matmul(h, w_hh)

tensor([1.2706, 1.1819, 1.3374, 0.7124])

In [ ]:
train_result